# Examine the spline interpolations on the RSX cut planes

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
from sys import path
path.append('/Users/vonderlinden2/rsx_analysis/writing_to_vtk/source')
import structured_vtk
reload(structured_vtk)

ImportError: No module named structured_vtk

# Look at Magnetic Field interpolations

In [3]:
bx_z357_dict = structured_vtk.read_data('data/bx_z357.sav')

NameError: name 'structured_vtk' is not defined

# Average at points that have mutiple measurements

In [4]:
data_dict = bx_z357_dict
data_dict['x_out'] = data_dict['x_out'].astype('float64')
data_dict['y_out'] = data_dict['y_out'].astype('float64')
data_dict['a_out'] = data_dict['a_out'].astype('float64')
time_points = data_dict['a_out'].shape[0]
data = {}
for idx in xrange(data_dict['x_out'].size):
    location = (data_dict['x_out'][idx], data_dict['y_out'][idx])
    if location in data.keys():
         data[location] = np.column_stack((data[location], data_dict['a_out'][:, idx]))
    else:
        data[location] = data_dict['a_out'][:, idx] 

unique_data_dict = {'x_out': [],
                    'y_out': [],
                    'a_out': [],
                    'std': []}
for location in data.keys():
    if data[location][0].size > 1:
        unique_data_dict['std'].append(data[location].std(axis=1, ddof=1))
        unique_data_dict['a_out'].append(data[location].mean(axis=1))  
    else:
        unique_data_dict['std'].append(np.zeros(time_points))
        unique_data_dict['a_out'].append(data[location])  
    unique_data_dict['x_out'].append(location[0])
    unique_data_dict['y_out'].append(location[1])

unique_data_dict['x_out'] = np.asarray(unique_data_dict['x_out'])
unique_data_dict['y_out'] = np.asarray(unique_data_dict['y_out'])
test = np.asarray(unique_data_dict['a_out'])
unique_data_dict['a_out'] = np.hsplit(np.asarray(unique_data_dict['a_out']), time_points)
unique_data_dict['delays'] = data_dict['delays']

to_remove = []
for idx in xrange(unique_data_dict['x_out'].size):
    if (-0.06 > unique_data_dict['x_out'][idx] or unique_data_dict['x_out'][idx] > 0.04 or 
       -0.02 > unique_data_dict['y_out'][idx] or unique_data_dict['y_out'][idx] > 0.025):
        to_remove.append(idx)
     
rect_a_out = []
for time_point in xrange(time_points):
    rect_a_out.append(np.delete(unique_data_dict['a_out'][time_point], to_remove))
rect_a_out = np.asarray(rect_a_out)

rect_data = {'x_out': np.delete(unique_data_dict['x_out'], to_remove),
             'y_out': np.delete(unique_data_dict['y_out'], to_remove),
             'a_out': rect_a_out,
             'std': np.delete(unique_data_dict['std'], to_remove)}

NameError: name 'bx_z357_dict' is not defined

# Functions for bivariate spline interpolation with Fitpack

In [17]:
def spline_fit(data_dict, kx=1, ky=1, smooth_factor=None, x_points=30, y_points=30, knots=None, time_point=1):
    (x_min, x_max,
     y_min, y_max) = structured_vtk.determine_sample_bounds([data_dict])
    spline = structured_vtk.fit_bivariate_splines(data_dict, time_point,
                                                  weigth=None, kx=kx, ky=ky,
                                                  s=smooth_factor)
    x_min, x_max, y_min, y_max = [-0.06, 0.04, -0.02, 0.025] 
    knots = spline.get_knots()
    (scalar_resampled,
     residual,
     x_grid,
     y_grid) = structured_vtk.evaluate_spline_on_structured_grid(spline,
                                                                 x_min, x_max,
                                                                 y_min, y_max,
                                                                 x_points,
                                                                 y_points)
    return scalar_resampled, residual, x_grid, y_grid, spline, knots

In [18]:
def plot_spline_data_knots(scalar_resampled, x_grid, y_grid, data_dict, knots,
                           time_point = 0, 
                           spline_color='blue', data_color='black', knots_color='red'):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_wireframe(x_grid, y_grid, scalar_resampled, colors=spline_color)
    ax.scatter(data_dict['x_out'], data_dict['y_out'], data_dict['a_out'][time_point], c=data_color)
    #ax.scatter(knots[0], knots[1], 0, c=knots_color, marker='^')
    ax.set_xlabel('x [m]')
    ax.set_ylabel('y [m]')
    plt.show()

In [30]:
(scalar_resampled, residual, 
 x_grid, y_grid, spline, knots) = spline_fit(rect_data, kx=1, ky=1, smooth_factor=0)

/Users/vonderlinden2/anaconda/lib/python2.7/site-packages/scipy/interpolate/fitpack2.py:1033: UserWarning: ier=3135
  warnings.warn(message)


In [28]:
plot_spline_data_knots(scalar_resampled, x_grid, y_grid, rect_data, knots)

In [31]:
scalar_resampled[np.where(scalar_resampled > 0.01)] = np.nan
scalar_resampled[np.where(scalar_resampled < -0.01)] = np.nan
plot_spline_data_knots(scalar_resampled, x_grid, y_grid, rect_data, knots)

/Users/vonderlinden2/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in less
  from ipykernel import kernelapp as app


# Radial Basis function interpolation 

In [119]:
from scipy.interpolate import Rbf

In [172]:
rbf = Rbf(rect_data['x_out'], rect_data['y_out'], rect_data['a_out'][0], function='cubic', smooth=0)

In [176]:
x_min, x_max, y_min, y_max = [-0.06, 0.04, -0.02, 0.025]
x_points = 30
y_points = 30
x_grid, y_grid = np.meshgrid(np.linspace(x_min, x_max, x_points), np.linspace(y_min, y_max, y_points))
rbf_values = rbf(x_grid, y_grid)

In [177]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(x_grid, y_grid, rbf_values, colors='blue')
ax.scatter(rect_data['x_out'], rect_data['y_out'], rect_data['a_out'][time_point], c='black')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
plt.show()

# Interp2d interpolation

In [175]:
from scipy.interpolate import interp2d

In [181]:
interp2d_func = interp2d(rect_data['x_out'], rect_data['y_out'], rect_data['a_out'][0], kind='quintic')

/Users/vonderlinden2/anaconda/lib/python2.7/site-packages/scipy/interpolate/fitpack.py:975: RuntimeWarning: A theoretically impossible result when finding a smoothing spline
with fp = s. Probable causes: s too small or badly chosen eps.
(abs(fp-s)/s>0.001)
	kx,ky=5,5 nx,ny=31,24 m=788 fp=0.000074 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))


# RegularGridInterpolator

In [182]:
from scipy.interpolate import RegularGridInterpolator

In [203]:
np.dstack((rect_data['x_out'], rect_data['y_out']))[0].shape

(788, 2)

In [210]:
points = np.dstack((rect_data['x_out'], rect_data['y_out']))[0]
grid_func = RegularGridInterpolator(points[0], rect_data['a_out'][0])

ValueError: There are 2 point arrays, but values has 1 dimensions

# LinearND Interpolater

In [211]:
from scipy.interpolate import LinearNDInterpolator

In [212]:
points.shape

(788, 2)

In [217]:
 rect_data['a_out'][0].shape

(788,)

In [219]:
points = np.dstack((rect_data['x_out'], rect_data['y_out']))[0]
linearnd_func = LinearNDInterpolator(points, rect_data['a_out'][0])

In [228]:
x_min, x_max, y_min, y_max = [-0.06, 0.04, -0.02, 0.025]
x_points = 100
y_points = 100
x_grid, y_grid = np.meshgrid(np.linspace(x_min, x_max, x_points), np.linspace(y_min, y_max, y_points))
linearnd_values = linearnd_func(x_grid, y_grid)

In [229]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(x_grid, y_grid, linearnd_values, colors='blue')
ax.scatter(rect_data['x_out'], rect_data['y_out'], rect_data['a_out'][time_point], c='black')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
plt.show()

# CoughTocher2d

In [239]:
from scipy.interpolate import CloughTocher2DInterpolator

In [240]:
points = np.dstack((rect_data['x_out'], rect_data['y_out']))[0]
ct_func = CloughTocher2DInterpolator(points, rect_data['a_out'][0], rescale=True)

In [241]:
x_min, x_max, y_min, y_max = [-0.06, 0.04, -0.02, 0.025]
x_points = 100
y_points = 100
x_grid, y_grid = np.meshgrid(np.linspace(x_min, x_max, x_points), np.linspace(y_min, y_max, y_points))
ct_values = ct_func(x_grid, y_grid)

In [242]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(x_grid, y_grid, ct_values, colors='blue')
ax.scatter(rect_data['x_out'], rect_data['y_out'], rect_data['a_out'][time_point], c='black')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
plt.show()

# LinearND Interpolator

In [247]:
points = np.dstack((unique_data_dict['x_out'], unique_data_dict['y_out']))[0]
linearnd_func = LinearNDInterpolator(points, unique_data_dict['a_out'][0])

In [248]:
x_min, x_max, y_min, y_max = (unique_data_dict['x_out'].min(), 
                              unique_data_dict['x_out'].max(),
                              unique_data_dict['y_out'].min(),
                              unique_data_dict['y_out'].max())
x_points = 100
y_points = 100
x_grid, y_grid = np.meshgrid(np.linspace(x_min, x_max, x_points), np.linspace(y_min, y_max, y_points))
linearnd_values = linearnd_func(x_grid, y_grid)

In [249]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(x_grid, y_grid, linearnd_values, colors='blue')
ax.scatter(rect_data['x_out'], rect_data['y_out'], rect_data['a_out'][time_point], c='black')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
plt.show()

ValueError: too many values to unpack